## 1. Configuração Inicial

In [ ]:
# Instalação das bibliotecas (execute apenas uma vez)
!pip install openai numpy scipy matplotlib scikit-learn python-dotenv -q

In [ ]:
import warnings
from dotenv import load_dotenv
from openai import OpenAI

# Importando funções do módulo local
from embedding_utils import (
    get_embedding,
    cosine_similarity,
    plot_embeddings_2d,
    buscar_afirmacao_mais_relevante,
    mostrar_busca,
)

warnings.filterwarnings("ignore")

# Carrega variáveis do arquivo .env
load_dotenv()

print("✅ Bibliotecas importadas com sucesso!")

In [ ]:
# A chave da API é carregada automaticamente do arquivo .env
client = OpenAI()

print("✅ Cliente OpenAI configurado!")

## 2. Base de Conhecimento

Vamos criar uma base de conhecimento com afirmações sobre diversos temas.

In [ ]:
# Base de conhecimento: afirmações/fatos
afirmacoes = [
    # Geografia (índices 0-5)
    "A capital do Brasil é Brasília",
    "A Torre Eiffel fica em Paris",
    "O Rio Amazonas é o maior rio do mundo em volume de água",
    "Tóquio é a capital do Japão",
    "A Muralha da China tem mais de 20 mil quilômetros",
    "O Monte Everest é a montanha mais alta do mundo",
    # Natureza/Ciência (índices 6-11)
    "A casca da banana é amarela",
    "A água ferve a 100 graus Celsius",
    "O sol é uma estrela",
    "O coração humano tem quatro câmaras",
    "A luz viaja a 300 mil quilômetros por segundo",
    "O DNA contém as informações genéticas dos seres vivos",
    # Pessoas/Cultura (índices 12-16)
    "Messi nasceu na Argentina",
    "Leonardo da Vinci pintou a Mona Lisa",
    "Cristiano Ronaldo é Português",
    "Beethoven compôs a Nona Sinfonia",
    "Shakespeare escreveu Romeu e Julieta",
    # Tecnologia (índices 17-19)
    "Python é uma linguagem de programação",
    "O primeiro computador ocupava uma sala inteira",
    "A internet foi criada na década de 1960",
]

print("📚 Base de Conhecimento:")
print("=" * 60)
for i, afirmacao in enumerate(afirmacoes, 1):
    print(f"{i:2}. {afirmacao}")

## 3. Gerando Embeddings das Afirmações

In [ ]:
# Gerando embeddings para todas as afirmações
print("Gerando embeddings das afirmações...\n")

embeddings_afirmacoes = []
for afirmacao in afirmacoes:
    emb = get_embedding(client, afirmacao)
    embeddings_afirmacoes.append(emb)
    print(f"✓ {afirmacao[:40]}...")

print(f"\n✅ {len(embeddings_afirmacoes)} embeddings de frases gerados!")

In [ ]:
# Verificando a dimensão do embedding
print(f"Dimensão de cada embedding: {len(embeddings_afirmacoes[0])}")

## 4. Visualização 2D das Afirmações

Vamos ver como as afirmações se distribuem no espaço vetorial usando t-SNE.

In [ ]:
# Categorias para colorir (baseado no tema da afirmação)
categorias = {
    "Geografia": [0, 1, 2, 3, 4, 5],
    "Natureza/Ciência": [6, 7, 8, 9, 10, 11],
    "Pessoas/Cultura": [12, 13, 14, 15, 16],
    "Tecnologia/Outros": [17, 18, 19],
}

cores_map = {
    "Geografia": "blue",
    "Natureza/Ciência": "green",
    "Pessoas/Cultura": "red",
    "Tecnologia/Outros": "purple",
}

# Criando lista de cores para cada afirmação
cores_afirmacoes = ["gray"] * len(afirmacoes)
for categoria, indices in categorias.items():
    for idx in indices:
        cores_afirmacoes[idx] = cores_map[categoria]

# Configuração da legenda
legenda_afirmacoes = [
    {"color": "blue", "label": "Geografia"},
    {"color": "green", "label": "Natureza/Ciência"},
    {"color": "red", "label": "Pessoas/Cultura"},
    {"color": "purple", "label": "Tecnologia/Outros"},
]

# Usando a função de visualização 2D
embeddings_afirmacoes_2d = plot_embeddings_2d(
    palavras=[a[:30] + "..." if len(a) > 30 else a for a in afirmacoes],
    embeddings_list=embeddings_afirmacoes,
    cores=cores_afirmacoes,
    legenda_config=legenda_afirmacoes,
    perplexity=5,
    figsize=(16, 12),
)

print("\n🎯 Observe como afirmações de temas similares tendem a se agrupar!")

## 5. Testando a Busca Semântica

Vamos fazer perguntas e ver qual afirmação é mais relevante para cada uma!

In [ ]:
# Teste 1: Pergunta sobre geografia
mostrar_busca(client, "Qual é a capital do Brasil?", afirmacoes, embeddings_afirmacoes)

In [ ]:
# Teste 2: Pergunta sobre esporte
mostrar_busca(client, "Onde o Messi nasceu?", afirmacoes, embeddings_afirmacoes)

In [ ]:
# Teste 3: Pergunta sobre ciência
mostrar_busca(
    client,
    "A que temperatura a água entra em ebulição?",
    afirmacoes,
    embeddings_afirmacoes,
)

In [ ]:
# Teste 4: Pergunta sobre frutas (formulada de forma diferente)
mostrar_busca(client, "De que cor é uma banana?", afirmacoes, embeddings_afirmacoes)

In [ ]:
# Teste 5: Pergunta sobre arte
mostrar_busca(client, "Quem pintou a Mona Lisa?", afirmacoes, embeddings_afirmacoes)

In [ ]:
# Teste 6: Pergunta em linguagem bem diferente da afirmação
mostrar_busca(
    client,
    "Qual é o pico mais elevado do planeta Terra?",
    afirmacoes,
    embeddings_afirmacoes,
)

In [ ]:
mostrar_busca(
    client,
    "Como é dividido nosso principal músculo que bombeia sangue?",
    afirmacoes,
    embeddings_afirmacoes,
)

In [ ]:
mostrar_busca(
    client,
    "Qual o tamanho das máquinas de computação primordias?",
    afirmacoes,
    embeddings_afirmacoes,
)

## 6. Experimente você mesmo!

Faça suas próprias perguntas e veja qual afirmação o sistema encontra!

In [ ]:
# Digite sua pergunta aqui!
mostrar_busca(client, "Onde fica a Torre Eiffel?", afirmacoes, embeddings_afirmacoes)

## 💡 O que aprendemos?

1. **Busca semântica funciona!** - O sistema encontra a afirmação correta mesmo quando a pergunta usa palavras diferentes
2. **Não é busca por palavras-chave** - "pico mais elevado" encontrou "montanha mais alta" 
3. **Base para RAG** - Isso é exatamente o que sistemas RAG fazem: encontram informações relevantes para dar contexto ao LLM

### 🚀 Próximo passo: RAG completo

No próximo notebook, vamos combinar:
- **Busca semântica** (o que fizemos aqui)
- **LLM** (para gerar respostas)
- **Contexto** (as afirmações encontradas)

Isso é o **RAG (Retrieval Augmented Generation)**!